In [2]:
import pandas as pd
import numpy as n
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt
from pprint import pprint

In [4]:
ufc1_file = "Resources/data.csv"
ufc1_df = pd.read_csv(ufc1_file)
ufc1_df.head()

,R_fighter,B_fighter,R_odds,B_odds,date,location,country,Winner,title_bout,weight_class,gender
0,Kevin Lee,Charles Oliveira,-145.0,135.0,3/14/2020,"Brasilia, Distrito Federal, Brazil",Brazil,Blue,False,Lightweight,MALE
1,Demian Maia,Gilbert Burns,160.0,-170.0,3/14/2020,"Brasilia, Distrito Federal, Brazil",Brazil,Blue,False,Welterweight,MALE
2,Renato Moicano,Damir Hadzovic,-345.0,315.0,3/14/2020,"Brasilia, Distrito Federal, Brazil",Brazil,Red,False,Lightweight,MALE
3,Johnny Walker,Nikita Krylov,-137.0,127.0,3/14/2020,"Brasilia, Distrito Federal, Brazil",Brazil,Blue,False,Light Heavyweight,MALE
4,Francisco Trinaldo,John Makdessi,-160.0,150.0,3/14/2020,"Brasilia, Distrito Federal, Brazil",Brazil,Red,False,Lightweight,MALE


In [6]:
ufc2_file = "Resources/ufc-master.csv"
ufc2_df = pd.read_csv(ufc2_file)
ufc2_df.head()

,R_fighter,B_fighter,R_odds,B_odds,R_ev,B_ev,date,location,country,Winner,...,R_td_attempted_bout,B_td_attempted_bout,R_td_pct_bout,B_td_pct_bout,R_sub_attempts_bout,B_sub_attempts_bout,R_pass_bout,B_pass_bout,R_rev_bout,B_rev_bout
0,Anthony Smith,Devin Clark,-139,115,71.942446,115.0,11/28/2020,"Las Vegas, Nevada, USA",USA,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Miguel Baeza,Takashi Sato,-175,140,57.142857,140.0,11/28/2020,"Las Vegas, Nevada, USA",USA,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Josh Parisian,Parker Porter,-200,160,50.000000,160.0,11/28/2020,"Las Vegas, Nevada, USA",USA,Blue,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Spike Carlyle,Bill Algeo,-177,140,56.497175,140.0,11/28/2020,"Las Vegas, Nevada, USA",USA,Blue,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Ashlee Evans-Smith,Norma Dumont,-137,110,72.992701,110.0,11/28/2020,"Las Vegas, Nevada, USA",USA,Blue,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
# Create a filtered dataframe from specific columns
ufc1_cols = ["date", "R_fighter", "B_fighter", "R_odds", "B_odds", "Winner"]
ufc1_transformed= ufc1_df[ufc1_cols].copy()

# Rename the column headers
ufc1_transformed = ufc1_transformed.rename(columns={"date": "Date",
                                                    "R_fighter": "Red_Corner_Fighter",
                                                    "B_fighter": "Blue_Corner_Fighter",
                                                    "R_odds": "Red_Fighter_odds",
                                                    "B_odds": "Blue_Fighter_odds",
                                                    "Winner": "Winner"})

# Clean the data by dropping duplicates and setting the index
#ufc1_transformed.drop_duplicates("Winner", inplace=True)
#ufc1_transformed.set_index("Winner", inplace=True)


ufc1_transformed.head()

,Date,Red_Corner_Fighter,Blue_Corner_Fighter,Red_Fighter_odds,Blue_Fighter_odds,Winner
0,3/14/2020,Kevin Lee,Charles Oliveira,-145.0,135.0,Blue
1,3/14/2020,Demian Maia,Gilbert Burns,160.0,-170.0,Blue
2,3/14/2020,Renato Moicano,Damir Hadzovic,-345.0,315.0,Red
3,3/14/2020,Johnny Walker,Nikita Krylov,-137.0,127.0,Blue
4,3/14/2020,Francisco Trinaldo,John Makdessi,-160.0,150.0,Red


In [30]:
ufc1_transformed.describe()

,Red_Fighter_odds,Blue_Fighter_odds
count,4240.000000,4240.000000
mean,-118.092217,68.525708
std,272.796068,252.371462
min,-1700.000000,-1200.000000
25%,-256.250000,-145.000000
50%,-150.000000,130.000000
75%,130.000000,225.000000
max,775.000000,1300.000000


In [46]:
ufc2_cols = ["date", "R_fighter", "B_fighter", "R_odds", "B_odds", "Winner", "R_ev", "B_ev"]
ufc2_transformed = ufc2_df[ufc2_cols].copy()

# Rename the column headers
ufc2_transformed = ufc2_transformed.rename(columns={"date": "Date",
                                                    "R_fighter": "Red_Corner_Fighter",
                                                    "B_fighter": "Blue_Corner_Fighter",
                                                    "R_odds": "Red_Fighter_odds",
                                                    "B_odds": "Blue_Fighter_odds",
                                                    "Winner": "Winner",
                                                    "R_ev": "Red_profit_on_a_100_credit_winnin_bet",
                                                    "B_ev": "Blue_profit_on_a_100_credit_winnin_bet"})

# Set index
#ufc2_transformed.set_index("Winner", inplace=True)

ufc2_transformed.head()

,Date,Red_Corner_Fighter,Blue_Corner_Fighter,Red_Fighter_odds,Blue_Fighter_odds,Winner,Red_profit_on_a_100_credit_winnin_bet,Blue_profit_on_a_100_credit_winnin_bet
0,11/28/2020,Anthony Smith,Devin Clark,-139,115,Red,71.942446,115.0
1,11/28/2020,Miguel Baeza,Takashi Sato,-175,140,Red,57.142857,140.0
2,11/28/2020,Josh Parisian,Parker Porter,-200,160,Blue,50.000000,160.0
3,11/28/2020,Spike Carlyle,Bill Algeo,-177,140,Blue,56.497175,140.0
4,11/28/2020,Ashlee Evans-Smith,Norma Dumont,-137,110,Blue,72.992701,110.0


In [31]:
ufc2_transformed.describe()

,Red_Fighter_odds,Blue_Fighter_odds,Red_profit_on_a_100_credit_winnin_bet,Blue_profit_on_a_100_credit_winnin_bet
count,4495.000000,4495.000000,4495.000000,4495.000000
mean,-119.595106,68.329032,95.054380,169.258737
std,271.780989,250.507730,84.185164,139.191233
min,-1700.000000,-1200.000000,5.882353,8.333333
25%,-260.000000,-145.000000,38.461538,68.965517
50%,-150.000000,130.000000,66.666667,130.000000
75%,129.500000,225.000000,129.500000,225.000000
max,775.000000,1300.000000,775.000000,1300.000000


In [22]:
connection_string = "postgres:postgres@localhost:5432/ufc_db"
engine = create_engine(f'postgresql://{connection_string}')

In [23]:
# Confirm tables
engine.table_names()

[]

In [24]:
ufc1_transformed.to_sql(name='ufc1', con=engine, if_exists='append', index=True)

In [25]:
ufc2_transformed.to_sql(name='ufc2', con=engine, if_exists='append', index=True)

In [47]:
##Merge ufc1_transformed and ufc2_transformed
ufc_all = pd.merge(ufc1_transformed, ufc2_transformed, on="Date")
ufc_all

,Date,Red_Corner_Fighter_x,Blue_Corner_Fighter_x,Red_Fighter_odds_x,Blue_Fighter_odds_x,Winner_x,Red_Corner_Fighter_y,Blue_Corner_Fighter_y,Red_Fighter_odds_y,Blue_Fighter_odds_y,Winner_y,Red_profit_on_a_100_credit_winnin_bet,Blue_profit_on_a_100_credit_winnin_bet
0,3/14/2020,Kevin Lee,Charles Oliveira,-145.0,135.0,Blue,Kevin Lee,Charles Oliveira,-145,135,Blue,68.965517,135.000000
1,3/14/2020,Kevin Lee,Charles Oliveira,-145.0,135.0,Blue,Demian Maia,Gilbert Burns,160,-170,Blue,160.000000,58.823529
2,3/14/2020,Kevin Lee,Charles Oliveira,-145.0,135.0,Blue,Renato Moicano,Damir Hadzovic,-345,315,Red,28.985507,315.000000
3,3/14/2020,Kevin Lee,Charles Oliveira,-145.0,135.0,Blue,Johnny Walker,Nikita Krylov,-137,127,Blue,72.992701,127.000000
4,3/14/2020,Kevin Lee,Charles Oliveira,-145.0,135.0,Blue,Francisco Trinaldo,John Makdessi,-160,150,Red,62.500000,150.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
49799,3/21/2010,Eric Schafer,Jason Brilz,140.0,-160.0,Blue,Duane Ludwig,Darren Elkins,-155,135,Blue,64.516129,135.000000
49800,3/21/2010,Eric Schafer,Jason Brilz,140.0,-160.0,Blue,John Howard,Daniel Roberts,-210,175,Red,47.619048,175.000000
49801,3/21/2010,Eric Schafer,Jason Brilz,140.0,-160.0,Blue,Brendan Schaub,Chase Gormley,-260,220,Red,38.461538,220.000000
49802,3/21/2010,Eric Schafer,Jason Brilz,140.0,-160.0,Blue,Mike Pierce,Julio Paulino,-420,335,Red,23.809524,335.000000


In [50]:
ufc_all = ufc_all.rename(columns={"Red_Corner_Fighter_x": "Red_Corner_Fighter", "Blue_Corner_Fighter_x": "Blue_Corner_Fighter", "Red_Fighter_odds_x": "Red_Fighter_odds",
                                      "Blue_Fighter_odds_x": "Blue_Fighter_odds", "Winner_x": "Winner", "Red_Corner_Fighter_y": "Red_Corner_Fighter", "Blue_Corner_Fighter_y": "Blue_Corner_Fighter", "Red_Fighter_odds_y": "Red_Fighter_odds",
                                      "Blue_Fighter_odds_y": "Blue_Fighter_odds", "Winner_y": "Winner"})

ufc_all = ufc_all.rename(columns={"Red_Corner_Fighter_y": "Red_Corner_Fighter", "Blue_Corner_Fighter_y": "Blue_Corner_Fighter", "Red_Fighter_odds_y": "Red_Fighter_odds",
                                      "Blue_Fighter_odds_y": "Blue_Fighter_odds", "Winner_y": "Winner"})

ufc_all

,Date,Red_Corner_Fighter,Blue_Corner_Fighter,Red_Fighter_odds,Blue_Fighter_odds,Winner,Red_Corner_Fighter,Blue_Corner_Fighter,Red_Fighter_odds,Blue_Fighter_odds,Winner,Red_profit_on_a_100_credit_winnin_bet,Blue_profit_on_a_100_credit_winnin_bet
0,3/14/2020,Kevin Lee,Charles Oliveira,-145.0,135.0,Blue,Kevin Lee,Charles Oliveira,-145,135,Blue,68.965517,135.000000
1,3/14/2020,Kevin Lee,Charles Oliveira,-145.0,135.0,Blue,Demian Maia,Gilbert Burns,160,-170,Blue,160.000000,58.823529
2,3/14/2020,Kevin Lee,Charles Oliveira,-145.0,135.0,Blue,Renato Moicano,Damir Hadzovic,-345,315,Red,28.985507,315.000000
3,3/14/2020,Kevin Lee,Charles Oliveira,-145.0,135.0,Blue,Johnny Walker,Nikita Krylov,-137,127,Blue,72.992701,127.000000
4,3/14/2020,Kevin Lee,Charles Oliveira,-145.0,135.0,Blue,Francisco Trinaldo,John Makdessi,-160,150,Red,62.500000,150.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
49799,3/21/2010,Eric Schafer,Jason Brilz,140.0,-160.0,Blue,Duane Ludwig,Darren Elkins,-155,135,Blue,64.516129,135.000000
49800,3/21/2010,Eric Schafer,Jason Brilz,140.0,-160.0,Blue,John Howard,Daniel Roberts,-210,175,Red,47.619048,175.000000
49801,3/21/2010,Eric Schafer,Jason Brilz,140.0,-160.0,Blue,Brendan Schaub,Chase Gormley,-260,220,Red,38.461538,220.000000
49802,3/21/2010,Eric Schafer,Jason Brilz,140.0,-160.0,Blue,Mike Pierce,Julio Paulino,-420,335,Red,23.809524,335.000000
